<a href="https://colab.research.google.com/github/Carllllllll/Proposal/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 30.97 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 40.0 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-07-21 02:22:32--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (b

In [3]:
import os
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

os.getcwd()
os.chdir('/content')

In [8]:
df1 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2019_1.xlsx', index_col=None, header=0)
df2 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2019_2.xlsx', index_col=None, header=0)
df3 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2020_1.xlsx', index_col=None, header=0)
df4 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2020_2.xlsx', index_col=None, header=0)
df5 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2021_1.xlsx', index_col=None, header=0)
df6 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2021_2.xlsx', index_col=None, header=0)
df7 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2022_1.xlsx', index_col=None, header=0)

In [9]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [14]:
df

,제안번호,제안일,제안자,제안자사번,제안자 직책,부문,지역본부,부서,팀,팀코드,...,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,73451.0,2019-06-30 14:14:14,서혜민,311363,6급사원,영업,부산경남지역본부,울산사업부,중부지점,HDMF_483500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73402.0,2019-06-28 21:15:41,최영희,310055,5급사원,영업,대구경북지역본부,포항사업부,포항남지점,HDMF_740200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73421.0,2019-06-28 20:54:16,이병훈,303331,수석,보상,지방권보상본부,대구대인보상부,구미대인보상센터,HDMF_044532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73415.0,2019-06-28 20:46:32,정현주,307063,주임,영업,중부지역본부,천안사업부,천안중앙지점,HDMF_770300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73400.0,2019-06-28 20:14:48,오진실,310892,5급사원,영업,호남지역본부,동광주사업부,광주제일지점,HDMF_3M0100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21016,87715.0,2022-01-03 15:02:53,전혜인,312467.0,전임(5급),영업,개인영업부문,교차사업부,강남교차지점,HDMF_2Q0200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21017,87698.0,2022-01-03 14:17:50,김현,308466.0,차장,보상,장기보상본부,영남권장기손사부,영남권배상재물센터,HDMF_045903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21018,87671.0,2022-01-03 13:45:06,김다운,312442.0,전임(5급),영업,경인지역본부,수원사업부,수원미래지점,HDMF_3U0600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21019,87735.0,2022-01-03 10:23:22,박시현,311358.0,전임(5급),영업,호남지역본부,동광주사업부,상무제일지점,HDMF_3M0500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# list(df)
df['제안종류'].drop_duplicates()

0        타부서제안
23      자기업무제안
139       혁신제안
2073       NaN
4702      테마제안
Name: 제안종류, dtype: object

In [15]:
df = df.loc[df['제안종류'] == '타부서제안']

In [16]:
def remove_basic(tmp) :
  tmp = tmp.replace('1. 제안배경', '')
  tmp = tmp.replace('* 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성 ', '')

  tmp = tmp.replace('2. 개선안(제안내용)', '')
  tmp = tmp.replace('* 제안내용의 개선방안 또는 개선가능 방향에 대해 작성 ', '')

  tmp = tmp.replace('3. 예상효과', '')
  tmp = tmp.replace('※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.', '')
  tmp = tmp.replace('(1) 재무 성과 (年 기준/천원 단위)', '')
  tmp = tmp.replace('(2) 업무 효율화 (年 기준)', '')
  tmp = tmp.replace('(3) 기타 효과', '')

  tmp = tmp.replace('   ', '')
  tmp = tmp.replace('  ', '')
  
  tmp = tmp.replace('* ', '')
  tmp = tmp.replace('*', '')
  return tmp

In [18]:
df['제안내용'] = df['제안내용'].astype(str).apply(remove_basic)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
df.dropna

<bound method DataFrame.dropna of           제안번호                 제안일  제안자     제안자사번  제안자 직책  부문      지역본부  \
0      73451.0 2019-06-30 14:14:14  서혜민    311363    6급사원  영업  부산경남지역본부   
1      73402.0 2019-06-28 21:15:41  최영희    310055    5급사원  영업  대구경북지역본부   
2      73421.0 2019-06-28 20:54:16  이병훈    303331      수석  보상   지방권보상본부   
3      73415.0 2019-06-28 20:46:32  정현주    307063      주임  영업    중부지역본부   
4      73400.0 2019-06-28 20:14:48  오진실    310892    5급사원  영업    호남지역본부   
...        ...                 ...  ...       ...     ...  ..       ...   
21016  87715.0 2022-01-03 15:02:53  전혜인  312467.0  전임(5급)  영업    개인영업부문   
21017  87698.0 2022-01-03 14:17:50   김현  308466.0      차장  보상    장기보상본부   
21018  87671.0 2022-01-03 13:45:06  김다운  312442.0  전임(5급)  영업    경인지역본부   
21019  87735.0 2022-01-03 10:23:22  박시현  311358.0  전임(5급)  영업    호남지역본부   
21020  87618.0 2022-01-03 10:12:50  정하영  312730.0  전임(6급)  영업  대구경북지역본부   

             부서          팀          팀코드  ... Unnamed: 58 Unnamed:

In [21]:
# mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
mecab = Mecab()
tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    
    text = row['제안내용']
    tag = row['제목']
    if pd.isna(text): continue
    if pd.isna(tag): continue
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

100%|██████████| 18927/18927 [00:19<00:00, 978.80it/s] 

문서의 수 : 18927


In [81]:
tagged_corpus_list[0]

TaggedDocument(words=['장기', '이', '재력', '의', '경우', '플래너', '는', '개인', '정보', '보호', '로', '인해', '열람', '이', '제한', '되', '어', '있', '음', '그러나', '확인', '하', '고자', '한다면', ',', '총무', '를', '통해', '이', '재력', '을', '확인', '하', '고', '있', '어', '얼마', '든지', '열람', '이', '가능', '한', '상태', '임', '이러', '한', '과정', '은', '플래너', '총무', '를', '거친', '이중', '작업', '임', '플래너', '또한', '장기', '이', '재력', '열람', '이', '가능', '하', '고자', '전산', '변경', '요청', '하', '는', '바', '임', '(', '단', ',', '열람', '시', '플래너', '사', '번', ',', '비밀', '번호', ',', '열람', '하', '고자', '하', '는', '사유', '기재', ',', '필요시', '고객', '동의', '인증', '까지', ')', '총무', '를', '거친', '이중', '업무', '해소', '플래너', '불만', '감소'], tags=['장기 이재력 플래너 열람 가능'])

In [82]:
from gensim.models import doc2vec

In [24]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
# print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('/content/drive/MyDrive/Proposal/dart.doc2vec')

In [63]:
# df.iat[i, 1]
i = 10
str = df.at[i, '제목']
# str = '자동차 (신)청약서 보험료 납입 부분 개선 요청'
str = 'HI-S포탈 자동차 모집자서명 생성요청 '
print(str)

HI-S포탈 자동차 모집자서명 생성요청 


In [113]:
# 87655 하면 87630 잘 나옴
str = df.loc[df['제안번호'] == 88553, '제목']
print(str)

20014    자동차 청약서 개선
Name: 제목, dtype: object


In [122]:
similar_doc = model.docvecs.most_similar(str)
# print(similar_doc[0][0])
# print(similar_doc[1][0])
# print(similar_doc[2][0])
# print(similar_doc[3][0])
# print(df.loc[df['제목'] == similar_doc[0][0]])
print(df.loc[df['제목'] == similar_doc[0][0]]['제안번호'])
print(df.loc[df['제목'] == similar_doc[1][0]]['제안번호'])
print(df.loc[df['제목'] == similar_doc[2][0]]['제안번호'])
print(df.loc[df['제목'] == similar_doc[3][0]]['제안번호'])

16024    86888.0
Name: 제안번호, dtype: object
5497    75031
Name: 제안번호, dtype: object
20012    88557.0
Name: 제안번호, dtype: object
16538    86430.0
Name: 제안번호, dtype: object


In [29]:
str2 = similar_doc[1][0]

In [ ]:
df.loc[df['제목'] == str].iat[0,1]

'월 마감후 미납해지 계약을 플래너분들에게 서면으로 나눠주고 있는데, 발행키가 없어서 엑셀내려받기후 편집해서 나눠주고 있습니다.플래너별 조회후 발행키가 있어서 깔끔하게 출력되었음 합니다.플래너분들이 스스로 미납해지 계약을 조회하고 안내하는 경우가 별로 없어총무들이 엑셀내려받기후 플래너분들에게 나눠주고 안내후 부활하고자 하는데업무의 편의성이 증대될것 같습니다.'

In [ ]:
df.loc[df['제목'] == str2].iat[0,1]

'구실손 관리를 위해 구실손 관리계약리스트를 활용.플래너 개인별 리스트를 본인이 출력하길 원함.엑셀 내려받은 후 발행을 유도하였으나, 엑셀 정리를 어려워하심.구실손 관리계약리스트 내에 발행버튼이 생성되었으면 합니다.플래너 개별적으로 효율적인 구실손 관리를 할 수 있음. ※예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
similar_doc[1][0]

'구실손 관리계약리스트 내 발행버튼 생성 요청'

In [106]:
# 정확도 높은 건들 찾기

for i in range(1,len(df)):
    
    try:
        str = df.at[i, '제목']
        similar_doc = model.docvecs.most_similar(str)
    except:
        a = 3

    if similar_doc[1][1] > 0.8:
        print(similar_doc[1][0])
        print(similar_doc[1][1])
        print(str)

암진단코드 C77~C80 진단시 약관상 유의사항부분 상품설명서상에  자세히 기재  요청 및 플래너 재교육요청
0.9083530306816101
의료기관별 합의참고정보 활용 고도화 방안
증권증권 재발행 확대
0.9201424717903137
장기재발행 권한 확대
계속보험료 입금표 화면 팝업 개선
0.95406174659729
기업대출 출금거래시 하이미 전자결재 업무 삭제 제안
소급부활 지역입력 개선요청
0.9657549262046814
소급부활지연입력 개선요청
소급부활지연입력 개선요청
0.9738296270370483
소급계약 지연입력 개선요청
소급부활지연입력 개선요청
0.9657549262046814
소급부활 지역입력 개선요청
증권증권 재발행 확대
0.9140907526016235
장기 증권 재발행 요청
보상포탈 화면 '일정'에서 접수번호를 클릭하면 바로 '업무처리' 화면으로 갈 수 있도록 링크 개선 부탁드립니다. 
0.9566095471382141
지급보증서 발송 기준 개선
장기 신계약 전자서명대상건 - 청약서 출력 프로세스 개선
0.834805965423584
전자서명건 청약서 발행시 확정 프로세스 개선
장기 증권 재발행 요청
0.9140907526016235
증권증권 재발행 확대
청약건수 제한해제 프로세스 개선 요청
0.9602038264274597
콜철회접수건 일괄송금처리 요청
콜철회접수건 일괄송금처리 요청
0.9576188325881958
전자화작업장송부화면 개선
지급보증서 발송 기준 개선
0.957548975944519
암진단코드 C77~C80 진단시 약관상 유의사항부분 상품설명서상에  자세히 기재  요청 및 플래너 재교육요청
콜철회접수건 일괄송금처리 요청
0.9581365585327148
여행자보험 보험수익자변경동의서 청약서 개선 및 자동스캔인식 요청
조치대상 전산제안
0.887024998664856
조치대상 화면 제안
조치대상 전산제안
0.887024998664856
조치대상 화면 제안
조치대상 전산제안
0.887024998664856
조치대상 

In [ ]:
df.at[2250, '제안내용']

'1. 제안배경         * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성           매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.     오전/오후 나누어 보고있는데     종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고     다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     또한 후임총무와 함께 업무처리를 하게되면 한 플래너에게 이중연락등 중복되게됩니다.     2. 개선안(제안내용)        * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성            하단에 메모기능을 추가하여     메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.           3. 예상효과        ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.           (3) 기타 효과           시간감소, 효율적 업무처리 가능        ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
str = df.at[2250, '제목']
similar_doc = model.docvecs.most_similar(str)

In [ ]:
similar_doc

[('조치대상 화면 보완', 0.847792387008667),
 ('조치대상 화면 제안', 0.8416188955307007),
 ('대리점 조치대상화면 수정요청합니다.', 0.43335217237472534),
 ('콜적부 조치기한 알람 뜨도록 개선요청', 0.41208720207214355),
 ('플래너 본인 계약 시 모집자 서명 안해도 확정가능하도록 제안합니다. ', 0.39692389965057373),
 ('완전판매 모니터링 최종차수 조치내용 관련', 0.39235469698905945),
 ('플래너 본인계약 모집자서명 생략', 0.38509178161621094),
 ('완전판매 모니터링 대상 리스트 제공필요', 0.375488817691803),
 ('하이포탈에서 어린이보험 설계건 조회 요청', 0.3726857304573059),
 ('장기 계속보험료 입금 취소건 지점 전결 처리', 0.3718041181564331)]

In [ ]:
df.loc[df['제목'] == '조치대상 화면 보완'].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.  오전/오후 나누어 보고있는데  종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고  다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성      하단에 메모기능을 추가하여  메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.     3. 예상효과     ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.        (3) 기타 효과     시간감소, 효율적 업무처리 가능     ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
df.loc[df['제목'] == '조치대상 화면 제안'].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.  오전/오후 나누어 보고있는데  종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고  다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성      하단에 메모기능을 추가하여  메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.     3. 예상효과     ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.        (3) 기타 효과     시간감소, 효율적 업무처리 가능     ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'